In [2]:
import openmm
from openmm import app, unit
import urllib

In [3]:
urllib.request.urlretrieve(
    "https://files.rcsb.org/view/2Z7F.pdb",
    "2Z7F.pdb"
)

('2Z7F.pdb', <http.client.HTTPMessage at 0x107905790>)

### Structure preparation

In [21]:
from pdbfixer import PDBFixer
fixer = PDBFixer("2Z7F.pdb")
fixer.removeHeterogens(True)
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.addMissingHydrogens(7.0)

In [25]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

In [26]:
modeler = app.Modeller(fixer.topology, fixer.positions)
modeler.addHydrogens(forcefield=forcefield, pH=7.0)
modeler.addSolvent(forcefield=forcefield, padding=1.0*unit.nanometers)

For now on, you should be able to run the simulation and save the output to `output.pdb`,
following the instructions from previous notebook.

💡Hint:
Now the `topology` and `position` are _attributes_ of the `modeler`.

In [27]:
system = forcefield.createSystem(
    modeler.topology, 
    nonbondedMethod=app.PME,
    nonbondedCutoff=1*unit.nanometer, 
    constraints=app.HBonds,
)

In [29]:
integrator = openmm.LangevinIntegrator(
    300*unit.kelvin, 
    1.0/unit.picoseconds, 
    2.0*unit.femtoseconds,
)

In [30]:
simulation = app.Simulation(modeler.topology, system, integrator)

In [31]:
simulation.context.setPositions(modeler.positions)

In [32]:
simulation.minimizeEnergy()

In [33]:
simulation.reporters.append(app.PDBReporter('output.pdb', 100))

In [34]:
simulation.step(10000)

As usual, we we carry on to the analysis stage of the simulation.

In [35]:
import mdtraj as md
traj = md.load('output.pdb')